In [1]:
import re
import random
import itertools
from typing import List, Union, Set, Tuple, Dict, Any, Iterator
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
from dataclasses import dataclass

@dataclass
class ShapPerturbation:
    """Container for SHAP perturbation result."""
    coalition: Set[int]  # Indices of unmasked tokens
    masked_text: str     # Text with masked tokens
    coalition_size: int  # Size of the coalition
    mask_vector: List[bool]  # Boolean mask vector

class ShapDocumentPerturber:
    """
    SHAP-style document perturbation system that creates coalitions of tokens
    for computing Shapley values. Mimics the coalition sampling approach used
    in SHAP text explainers.
    """
    
    def __init__(self, mask_token: str = "<unk>", seed: int = 42):
        self.mask_token = mask_token
        self.seed = seed
        random.seed(seed)
        np.random.seed(seed)
        
        # Precompiled regex for tokenization
        self.word_pattern = re.compile(r'\b\w+\b')
    
    def tokenize_with_positions(self, text: str) -> List[Tuple[str, int, int]]:
        """
        Tokenize text and return tokens with their positions.
        Returns: List of (token, start_pos, end_pos)
        """
        tokens_with_pos = []
        for match in self.word_pattern.finditer(text):
            token = match.group().lower()
            start, end = match.span()
            tokens_with_pos.append((token, start, end))
        return tokens_with_pos
    
    def generate_all_coalitions(self, n_tokens: int) -> Iterator[Set[int]]:
        """Generate all possible coalitions (power set) of token indices."""
        for r in range(n_tokens + 1):
            for coalition in itertools.combinations(range(n_tokens), r):
                yield set(coalition)
    
    def sample_coalitions(self, n_tokens: int, 
                         n_samples: int = None,
                         include_empty_and_full: bool = True) -> List[Set[int]]:
        """
        Sample coalitions following SHAP's sampling strategy.
        
        Args:
            n_tokens: Number of tokens in the document
            n_samples: Number of coalitions to sample (None for all)
            include_empty_and_full: Always include empty and full coalitions
        
        Returns:
            List of coalition sets (token indices)
        """
        if n_samples is None or n_samples >= 2**n_tokens:
            # Return all coalitions if feasible
            return list(self.generate_all_coalitions(n_tokens))
        
        coalitions = set()
        
        # Always include empty and full coalitions (important for SHAP)
        if include_empty_and_full:
            coalitions.add(frozenset())  # Empty coalition
            coalitions.add(frozenset(range(n_tokens)))  # Full coalition
        
        # Sample remaining coalitions
        remaining_samples = n_samples - len(coalitions)
        
        while len(coalitions) < n_samples:
            # SHAP typically samples coalition sizes from a distribution
            # that gives more weight to smaller and larger coalitions
            coalition_size = self._sample_coalition_size(n_tokens)
            coalition = frozenset(random.sample(range(n_tokens), coalition_size))
            coalitions.add(coalition)
        
        return [set(coalition) for coalition in coalitions]
    
    def _sample_coalition_size(self, n_tokens: int) -> int:
        """
        Sample coalition size using SHAP's distribution.
        SHAP gives higher probability to smaller and larger coalitions.
        """
        # Use beta distribution to bias towards extremes
        # This mimics SHAP's coalition sampling strategy
        alpha, beta = 0.5, 0.5  # Parameters for beta distribution
        p = np.random.beta(alpha, beta)
        coalition_size = int(p * n_tokens)
        return max(0, min(coalition_size, n_tokens))
    
    def stratified_coalition_sampling(self, n_tokens: int, 
                                    n_samples: int,
                                    strata_sizes: List[int] = None) -> List[Set[int]]:
        """
        Stratified sampling of coalitions by size (like SHAP Explainer).
        
        Args:
            n_tokens: Number of tokens
            n_samples: Total number of samples
            strata_sizes: Number of samples per coalition size stratum
        
        Returns:
            List of sampled coalitions
        """
        if strata_sizes is None:
            # Default: sample proportionally to binomial distribution
            strata_sizes = self._compute_default_strata_sizes(n_tokens, n_samples)
        
        coalitions = []
        
        for size, n_size_samples in enumerate(strata_sizes):
            if n_size_samples == 0:
                continue
                
            if size == 0:
                # Empty coalition
                coalitions.append(set())
            elif size == n_tokens:
                # Full coalition
                coalitions.append(set(range(n_tokens)))
            else:
                # Sample coalitions of this size
                size_coalitions = []
                max_combinations = min(n_size_samples * 10, 
                                     int(np.math.comb(n_tokens, size)))
                
                attempts = 0
                while len(size_coalitions) < n_size_samples and attempts < max_combinations:
                    coalition = set(random.sample(range(n_tokens), size))
                    if coalition not in size_coalitions:
                        size_coalitions.append(coalition)
                    attempts += 1
                
                coalitions.extend(size_coalitions)
        
        return coalitions
    
    def _compute_default_strata_sizes(self, n_tokens: int, n_samples: int) -> List[int]:
        """Compute default stratified sample sizes."""
        # Allocate samples proportional to binomial coefficients
        # but cap very large strata
        strata_sizes = []
        total_weight = 0
        
        for size in range(n_tokens + 1):
            weight = min(np.math.comb(n_tokens, size), n_samples // 4)
            strata_sizes.append(weight)
            total_weight += weight
        
        # Normalize to n_samples
        if total_weight > 0:
            strata_sizes = [int(size * n_samples / total_weight) for size in strata_sizes]
        
        # Ensure we always have empty and full coalitions
        if n_tokens > 0:
            strata_sizes[0] = max(1, strata_sizes[0])  # Empty
            strata_sizes[-1] = max(1, strata_sizes[-1])  # Full
        
        return strata_sizes
    
    def create_perturbation(self, text: str, coalition: Set[int]) -> ShapPerturbation:
        """
        Create a single perturbation by masking tokens not in coalition.
        
        Args:
            text: Original text
            coalition: Set of token indices to keep unmasked
        
        Returns:
            ShapPerturbation object
        """
        tokens_with_pos = self.tokenize_with_positions(text)
        n_tokens = len(tokens_with_pos)
        
        if not tokens_with_pos:
            return ShapPerturbation(
                coalition=coalition,
                masked_text=text,
                coalition_size=0,
                mask_vector=[]
            )
        
        # Create mask vector
        mask_vector = [i in coalition for i in range(n_tokens)]
        
        # Create masked text by replacing tokens not in coalition
        masked_text = text
        offset = 0
        
        for i, (token, start, end) in enumerate(tokens_with_pos):
            if i not in coalition:
                # Replace token with mask token
                adjusted_start = start + offset
                adjusted_end = end + offset
                masked_text = (masked_text[:adjusted_start] + 
                             self.mask_token + 
                             masked_text[adjusted_end:])
                offset += len(self.mask_token) - (end - start)
        
        return ShapPerturbation(
            coalition=coalition,
            masked_text=masked_text,
            coalition_size=len(coalition),
            mask_vector=mask_vector
        )
    
    def generate_shap_samples(self, text: str, 
                            n_samples: int = None,
                            sampling_method: str = "random") -> List[ShapPerturbation]:
        """
        Generate SHAP-style perturbations of a document.
        
        Args:
            text: Input document text
            n_samples: Number of samples (None for all possible)
            sampling_method: 'random', 'stratified', or 'all'
        
        Returns:
            List of ShapPerturbation objects
        """
        tokens_with_pos = self.tokenize_with_positions(text)
        n_tokens = len(tokens_with_pos)
        
        if n_tokens == 0:
            return [ShapPerturbation(set(), text, 0, [])]
        
        # Generate coalitions based on sampling method
        if sampling_method == "all" or (n_samples is None and n_tokens <= 15):
            coalitions = list(self.generate_all_coalitions(n_tokens))
        elif sampling_method == "stratified":
            coalitions = self.stratified_coalition_sampling(n_tokens, n_samples or 1000)
        else:  # random
            coalitions = self.sample_coalitions(n_tokens, n_samples or 1000)
        
        # Create perturbations
        perturbations = []
        for coalition in coalitions:
            perturbation = self.create_perturbation(text, coalition)
            perturbations.append(perturbation)
        
        return perturbations
    
    def generate_shap_samples_parallel(self, documents: List[str],
                                     n_samples: int = None,
                                     sampling_method: str = "random",
                                     n_jobs: int = -1,
                                     use_processes: bool = True) -> List[List[ShapPerturbation]]:
        """
        Generate SHAP samples for multiple documents in parallel.
        
        Args:
            documents: List of document texts
            n_samples: Number of samples per document
            sampling_method: Sampling strategy
            n_jobs: Number of parallel jobs
            use_processes: Use processes vs threads
        
        Returns:
            List of perturbation lists (one per document)
        """
        if n_jobs == -1:
            n_jobs = mp.cpu_count()
        
        # Create partial function
        sample_func = lambda doc: self.generate_shap_samples(
            doc, n_samples, sampling_method
        )
        
        if use_processes:
            with ProcessPoolExecutor(max_workers=n_jobs) as executor:
                results = list(executor.map(sample_func, documents))
        else:
            with ThreadPoolExecutor(max_workers=n_jobs) as executor:
                results = list(executor.map(sample_func, documents))
        
        return results
    
    def create_explanation_dataset(self, text: str, 
                                 model_predictions: callable = None,
                                 n_samples: int = 1000) -> Dict[str, Any]:
        """
        Create a dataset suitable for SHAP explanation computation.
        
        Args:
            text: Original text to explain
            model_predictions: Function that takes text and returns prediction
            n_samples: Number of perturbation samples
        
        Returns:
            Dictionary with perturbations and predictions
        """
        perturbations = self.generate_shap_samples(text, n_samples, "stratified")
        
        dataset = {
            'original_text': text,
            'perturbations': perturbations,
            'coalition_matrix': np.array([p.mask_vector for p in perturbations]),
            'coalition_sizes': [p.coalition_size for p in perturbations],
            'masked_texts': [p.masked_text for p in perturbations]
        }
        
        if model_predictions is not None:
            dataset['predictions'] = [
                model_predictions(p.masked_text) for p in perturbations
            ]
        
        return dataset

# Utility functions for SHAP-style analysis
def create_binary_feature_matrix(perturbations: List[ShapPerturbation]) -> np.ndarray:
    """Create binary feature matrix for SHAP computation."""
    if not perturbations:
        return np.array([])
    
    n_samples = len(perturbations)
    n_features = len(perturbations[0].mask_vector)
    
    matrix = np.zeros((n_samples, n_features), dtype=int)
    for i, p in enumerate(perturbations):
        matrix[i] = p.mask_vector
    
    return matrix

def compute_shapley_kernel_weights(coalition_sizes: List[int], 
                                 n_features: int) -> np.ndarray:
    """Compute SHAP kernel weights for coalition sizes."""
    weights = np.zeros(len(coalition_sizes))
    
    for i, size in enumerate(coalition_sizes):
        if size == 0 or size == n_features:
            # Infinite weight for empty and full coalitions in theory
            # but we use large finite weight in practice
            weights[i] = 1000
        else:
            # SHAP kernel weight formula
            weights[i] = (n_features - 1) / (
                np.math.comb(n_features, size) * size * (n_features - size)
            )
    
    return weights

# Example usage and demonstration
if __name__ == "__main__":
    # Sample documents
    sample_docs = [
        "The quick brown fox jumps over the lazy dog.",
        "Machine learning algorithms require careful feature engineering.",
        "Natural language processing enables computers to understand human text."
    ]
    
    # Initialize SHAP perturber
    perturber = ShapDocumentPerturber(mask_token="<unk>")
    
    # Example 1: Generate SHAP samples for a single document
    print("=== SHAP Perturbations for Single Document ===")
    text = sample_docs[0]
    perturbations = perturber.generate_shap_samples(text, n_samples=20)
    
    print(f"Original: {text}")
    print(f"Generated {len(perturbations)} perturbations:")
    
    for i, p in enumerate(perturbations[:10]):  # Show first 10
        print(f"  Coalition {i} (size {p.coalition_size}): {p.masked_text}")
    
    # Example 2: Create explanation dataset
    print("\n=== SHAP Explanation Dataset ===")
    
    # Mock prediction function
    def mock_sentiment_model(text: str) -> float:
        """Mock sentiment analysis model."""
        positive_words = {'good', 'great', 'excellent', 'amazing', 'wonderful'}
        negative_words = {'bad', 'terrible', 'awful', 'horrible', 'poor'}
        
        words = set(text.lower().split())
        pos_score = len(words.intersection(positive_words))
        neg_score = len(words.intersection(negative_words))
        
        return (pos_score - neg_score) / max(len(words), 1)
    
    explanation_data = perturber.create_explanation_dataset(
        "This movie is really great and amazing!",
        model_predictions=mock_sentiment_model,
        n_samples=50
    )
    
    print(f"Created dataset with {len(explanation_data['perturbations'])} samples")
    print(f"Coalition matrix shape: {explanation_data['coalition_matrix'].shape}")
    print(f"Sample predictions: {explanation_data['predictions'][:5]}")
    
    # Example 3: Feature importance computation setup
    print("\n=== Feature Matrix and Weights for SHAP ===")
    feature_matrix = create_binary_feature_matrix(perturbations[:10])
    coalition_sizes = [p.coalition_size for p in perturbations[:10]]
    n_features = len(perturber.tokenize_with_positions(text))
    weights = compute_shapley_kernel_weights(coalition_sizes, n_features)
    
    print(f"Feature matrix shape: {feature_matrix.shape}")
    print(f"Coalition sizes: {coalition_sizes}")
    print(f"SHAP weights: {weights}")
    
    # Example 4: Parallel processing
    print("\n=== Parallel SHAP Sample Generation ===")
    all_perturbations = perturber.generate_shap_samples_parallel(
        sample_docs, n_samples=30, n_jobs=2
    )
    
    for i, doc_perturbations in enumerate(all_perturbations):
        print(f"Document {i+1}: {len(doc_perturbations)} perturbations generated")

=== SHAP Perturbations for Single Document ===
Original: The quick brown fox jumps over the lazy dog.
Generated 20 perturbations:
  Coalition 0 (size 2): <unk> <unk> brown fox <unk> <unk> <unk> <unk> <unk>.
  Coalition 1 (size 9): The quick brown fox jumps over the lazy dog.
  Coalition 2 (size 2): <unk> <unk> <unk> <unk> jumps over <unk> <unk> <unk>.
  Coalition 3 (size 5): The quick brown <unk> <unk> over <unk> <unk> dog.
  Coalition 4 (size 6): The quick <unk> fox jumps over the <unk> <unk>.
  Coalition 5 (size 3): <unk> quick <unk> <unk> jumps <unk> <unk> <unk> dog.
  Coalition 6 (size 8): The quick brown fox <unk> over the lazy dog.
  Coalition 7 (size 7): <unk> quick brown fox jumps over the <unk> dog.
  Coalition 8 (size 8): <unk> quick brown fox jumps over the lazy dog.
  Coalition 9 (size 4): The <unk> brown fox <unk> over <unk> <unk> <unk>.

=== SHAP Explanation Dataset ===


AttributeError: module 'numpy' has no attribute 'math'

In [2]:
from elasticsearch import Elasticsearch
import pprint

# Connect to local ES instance
es = Elasticsearch("http://localhost:9200")

# Custom analyzer demo index
index_name = "analyzer_demo"

# Delete old index
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Create index with two fields: one default, one custom analyzer
es.indices.create(
    index=index_name,
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "custom_english_no_stop": {
                        "type": "standard",
                        "stopwords": "_none_"  # disables stopword filtering
                    },
                    "custom_english_with_stop": {
                        "type": "standard",
                        "stopwords": "_english_"  # use built-in English stopwords
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "no_stop": {"type": "text", "analyzer": "custom_english_no_stop"},
                "with_stop": {"type": "text", "analyzer": "custom_english_with_stop"}
            }
        }
    }
)

print("✅ Created index with custom analyzers")

# === ANALYZE some texts ===

text = "Elasticsearch is a powerful search engine with support for full-text search and stopword removal."

# Analyze using no-stopword analyzer
resp_no_stop = es.indices.analyze(
    index=index_name,
    body={
        "analyzer": "custom_english_no_stop",
        "text": text
    }
)

# Analyze using with-stopword analyzer
resp_with_stop = es.indices.analyze(
    index=index_name,
    body={
        "analyzer": "custom_english_with_stop",
        "text": text
    }
)

# Print out tokens
print("\n🔍 Tokens WITHOUT stopword removal:")
pprint.pprint([t["token"] for t in resp_no_stop["tokens"]])

print("\n🛑 Tokens WITH stopword removal (English stopwords):")
pprint.pprint([t["token"] for t in resp_with_stop["tokens"]])


✅ Created index with custom analyzers

🔍 Tokens WITHOUT stopword removal:
['elasticsearch',
 'is',
 'a',
 'powerful',
 'search',
 'engine',
 'with',
 'support',
 'for',
 'full',
 'text',
 'search',
 'and',
 'stopword',
 'removal']

🛑 Tokens WITH stopword removal (English stopwords):
['elasticsearch',
 'powerful',
 'search',
 'engine',
 'support',
 'full',
 'text',
 'search',
 'stopword',
 'removal']


In [4]:


# Simulate a binary mask: e.g., keep only 'love' and 'product'
keep = ["love", "product"]
mask = np.array([1 if word in keep else 0 for word in feature_names])



In [5]:
mask

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [7]:
# Create a perturbed version: masked features replaced by background
perturbed_vector = np.where(mask == 1, instance_vector, background_vector)
perturbed_vector

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.62276601, 0.62276601, 0.        , 0.        , 0.        ,
       0.        ])

In [8]:


# Show side-by-side
import pandas as pd

df = pd.DataFrame({
    "feature": feature_names,
    "original_tfidf": instance_vector,
    "mask": mask,
    "perturbed_tfidf": perturbed_vector
})

df

feature  original_tfidf  mask  perturbed_tfidf
0   absolutely        0.000000     0         0.000000
1         ever        0.000000     0         0.000000
2   experience        0.000000     0         0.000000
3    fantastic        0.000000     0         0.000000
4           is        0.000000     0         0.000000
5         love        0.622766     1         0.622766
6      product        0.622766     1         0.622766
7      service        0.000000     0         0.000000
8          the        0.000000     0         0.000000
9         this        0.473630     0         0.000000
10       worst        0.000000     0         0.000000

In [11]:

df = df[(df["original_tfidf"] > 0) | (df["perturbed_tfidf"] > 0)]
print(df.sort_values("original_tfidf", ascending=False))

# Predict
original_pred = model.predict_proba([instance_vector])[0,1]
perturbed_pred = model.predict_proba([perturbed_vector])[0,1]
delta = original_pred - perturbed_pred

print(f"\nOriginal Prediction: {original_pred:.4f}")
print(f"Perturbed Prediction (only 'love' and 'product'): {perturbed_pred:.4f}")
print(f"Change in prediction (contribution of masked features): {delta:.4f}")

   feature  original_tfidf  mask  perturbed_tfidf
5     love        0.622766     1         0.622766
6  product        0.622766     1         0.622766
9     this        0.473630     0         0.000000

Original Prediction: 0.7161
Perturbed Prediction (only 'love' and 'product'): 0.7204
Change in prediction (contribution of masked features): -0.0042


In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

print(es.info())

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.010s]


{'name': 'a11a2100a8aa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gAf831kRS1GKH2EiTlG_ug', 'version': {'number': '8.13.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '16cc90cd2d08a3147ce02b07e50894bc060a4cbf', 'build_date': '2024-04-05T14:45:26.420424304Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [10]:
%run ES_class.py

In [11]:

# Example usage and comprehensive testing
# if __name__ == "__main__":
try:
    # Initialize the hybrid IR system with all components enabled
    # Note: Make sure Elasticsearch is running on localhost:9200
    hybrid_ir = HybridInformationRetrieval(
        es_host="http://localhost:9200",  # Full URL with scheme
        enable_sparse=True,
        enable_dense=True,
        enable_reranker=True,
        lazy_load_models=True
    )
except ConnectionError as e:
    print(f"Error connecting to Elasticsearch: {e}")
    print("Please ensure Elasticsearch is running on http://localhost:9200")
    print("You can start Elasticsearch using Docker:")
    print("docker run -d --name elasticsearch -p 9200:9200 -e 'discovery.type=single-node' -e 'xpack.security.enabled=false' elasticsearch:8.11.0")
    exit(1)
except Exception as e:
    print(f"Unexpected error: {e}")
    exit(1)

# Sample documents for testing
sample_documents = [
    {
        "id": "doc1",
        "content": "Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.",
        "metadata": {"category": "AI", "author": "John Doe"}
    },
    {
        "id": "doc2", 
        "content": "Deep learning neural networks have revolutionized computer vision and natural language processing.",
        "metadata": {"category": "Deep Learning", "author": "Jane Smith"}
    },
    {
        "id": "doc3",
        "content": "Information retrieval systems help users find relevant documents from large collections.",
        "metadata": {"category": "IR", "author": "Bob Johnson"}
    },
    {
        "id": "doc4",
        "content": "Elasticsearch is a distributed search and analytics engine built on Apache Lucene.",
        "metadata": {"category": "Search", "author": "Alice Wilson"}
    },
    {
        "id": "doc5",
        "content": "Sentence transformers create dense vector representations of text for semantic search.",
        "metadata": {"category": "NLP", "author": "Charlie Brown"}
    }
]


INFO:elastic_transport.transport:HEAD http://localhost:9200/ [status:200 duration:0.185s]
INFO:__main__:Successfully connected to Elasticsearch at http://localhost:9200
INFO:elastic_transport.transport:HEAD http://localhost:9200/hybrid_search_index [status:404 duration:0.002s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index [status:200 duration:0.145s]
INFO:__main__:Created index: hybrid_search_index


In [16]:

# Add documents to the index
hybrid_ir.add_documents(sample_documents)

query = "machine learning algorithms for search"

# Test 1: Full pipeline with intermediate outputs
print("="*80)
print("TEST 1: FULL HYBRID SEARCH WITH INTERMEDIATE OUTPUTS")
print("="*80)

full_output = hybrid_ir.hybrid_search(
    query, 
    sparse_top_k=10, 
    dense_top_k=10, 
    final_top_k=3, 
    return_intermediate=False
)
hybrid_ir.display_hybrid_output(full_output, query)

# Test 2: Disable dense search and rerun
print("\n" + "="*80)
print("TEST 2: SPARSE SEARCH ONLY (DENSE DISABLED)")
print("="*80)

hybrid_ir.disable_component(SearchComponent.DENSE)
sparse_only_output = hybrid_ir.hybrid_search(query, return_intermediate=True)
hybrid_ir.display_hybrid_output(sparse_only_output, query)


INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc1 [status:200 duration:0.022s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc2 [status:200 duration:0.006s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc3 [status:200 duration:0.003s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc4 [status:200 duration:0.005s]
INFO:elastic_transport.transport:PUT http://localhost:9200/hybrid_search_index/_doc/doc5 [status:200 duration:0.005s]
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_refresh [status:200 duration:0.006s]
INFO:__main__:Added 5 documents to index
INFO:__main__:Starting hybrid search for query: 'machine learning algorithms for search'
INFO:__main__:Enabled components: ['sparse', 'reranker']
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_search [status:200 durat

TEST 1: FULL HYBRID SEARCH WITH INTERMEDIATE OUTPUTS


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s]
INFO:__main__:Reranked and returned top 3 results
INFO:__main__:Disabled component: dense
INFO:__main__:Starting hybrid search for query: 'machine learning algorithms for search'
INFO:__main__:Enabled components: ['sparse', 'reranker']
INFO:elastic_transport.transport:POST http://localhost:9200/hybrid_search_index/_search [status:200 duration:0.011s]
INFO:__main__:Sparse search returned 4 results
INFO:__main__:Dense search skipped (disabled)
INFO:__main__:After deduplication: 4 unique results



COMPLETE HYBRID SEARCH PIPELINE FOR: 'machine learning algorithms for search'

RESULT SUMMARY:
  sparse_count: 4
  dense_count: 0
  deduplicated_count: 4
  final_count: 3

SPARSE SEARCH RESULTS (BM25)

1. Document ID: doc1
   Score: 3.3349 (sparse)
   Content: Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.
   --------------------------------------------------

2. Document ID: doc5
   Score: 2.3566 (sparse)
   Content: Sentence transformers create dense vector representations of text for semantic search.
   --------------------------------------------------

3. Document ID: doc2
   Score: 0.8814 (sparse)
   Content: Deep learning neural networks have revolutionized computer vision and natural language processing.
   --------------------------------------------------

4. Document ID: doc4
   Score: 0.8814 (sparse)
   Content: Elasticsearch is a distributed search and analytics engine built on Apache Lucene.
   -----------------

Batches: 100%|██████████| 1/1 [00:00<00:00, 105.37it/s]
INFO:__main__:Reranked and returned top 3 results



COMPLETE HYBRID SEARCH PIPELINE FOR: 'machine learning algorithms for search'

RESULT SUMMARY:
  sparse_count: 4
  dense_count: 0
  combined_count: 4
  deduplicated_count: 4
  final_count: 3

SPARSE SEARCH RESULTS (BM25)

1. Document ID: doc1
   Score: 3.3349 (sparse)
   Content: Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.
   --------------------------------------------------

2. Document ID: doc5
   Score: 2.3566 (sparse)
   Content: Sentence transformers create dense vector representations of text for semantic search.
   --------------------------------------------------

3. Document ID: doc2
   Score: 0.8814 (sparse)
   Content: Deep learning neural networks have revolutionized computer vision and natural language processing.
   --------------------------------------------------

4. Document ID: doc4
   Score: 0.8814 (sparse)
   Content: Elasticsearch is a distributed search and analytics engine built on Apache Lucene.


In [15]:
SearchComponent.DENSE

<SearchComponent.DENSE: 'dense'>

In [ ]:

# Test 3: Enable dense, disable reranker
print("\n" + "="*80)
print("TEST 3: SPARSE + DENSE ONLY (RERANKER DISABLED)")
print("="*80)

hybrid_ir.enable_component(SearchComponent.DENSE)
hybrid_ir.disable_component(SearchComponent.RERANKER)
no_rerank_output = hybrid_ir.hybrid_search(query, return_intermediate=True)
hybrid_ir.display_hybrid_output(no_rerank_output, query)

# Test 4: Single component searches
print("\n" + "="*80)
print("TEST 4: SINGLE COMPONENT SEARCHES")
print("="*80)

# Re-enable all components for single component tests
hybrid_ir.enable_component(SearchComponent.RERANKER)

sparse_results = hybrid_ir.search_with_single_component(query, SearchComponent.SPARSE, top_k=5)
hybrid_ir.display_results(sparse_results, title="SPARSE ONLY")

dense_results = hybrid_ir.search_with_single_component(query, SearchComponent.DENSE, top_k=5)
hybrid_ir.display_results(dense_results, title="DENSE ONLY")

# Test 5: Toggle components dynamically
print("\n" + "="*80)
print("TEST 5: DYNAMIC COMPONENT TOGGLING")
print("="*80)

print("Current enabled components:", [comp.value for comp in hybrid_ir.get_enabled_components()])

hybrid_ir.toggle_component(SearchComponent.SPARSE)
print("After toggling sparse:", [comp.value for comp in hybrid_ir.get_enabled_components()])

hybrid_ir.toggle_component(SearchComponent.SPARSE)  # Toggle back
print("After toggling sparse again:", [comp.value for comp in hybrid_ir.get_enabled_components()])

In [19]:
# ------------------------------------------------------------
# 0. Imports
# ------------------------------------------------------------
import numpy as np
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

# ------------------------------------------------------------
# 1. Synthetic corpus + labels
# ------------------------------------------------------------
positive_templates = [
    "I absolutely love this product",
    "Amazing experience and great quality",
    "Fantastic service, will come again",
    "Totally satisfied with the purchase",
    "Highly recommend to everyone"
]
negative_templates = [
    "I really hate this item",
    "Terrible customer service and quality",
    "Very disappointing experience overall",
    "I regret buying this product",
    "Not worth the money at all"
]

# Build a small corpus of 100 documents (50 pos / 50 neg)
random.seed(42)
docs = [random.choice(positive_templates) for _ in range(50)] + \
       [random.choice(negative_templates) for _ in range(50)]
labels = np.array([1]*50 + [0]*50)     # 1 = positive, 0 = negative

# ------------------------------------------------------------
# 2. TF‑IDF vectorisation
# ------------------------------------------------------------
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(docs)           # sparse CSR matrix
feature_names = vectorizer.get_feature_names_out() # list of token/feature names

# ------------------------------------------------------------
# 3. Train / test split + model
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, labels, test_size=0.2, random_state=42, stratify=labels
)

clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_train)

print("Hold‑out accuracy:",
      accuracy_score(y_test, clf.predict(X_test)))

# ------------------------------------------------------------
# 4. Build background data for KernelSHAP
#      – use a random 50‑sample subset from the *train* set
# ------------------------------------------------------------
rng = np.random.default_rng(0)
bg_idx = rng.choice(X_train.shape[0], size=50, replace=False)
background_tfidf = X_train[bg_idx].toarray()   # convert to dense for SHAP
X_explain    = X_test[:3].toarray()            # explain first 3 test points
# assuming you already have: clf, background_tfidf, X_explain, feature_names

import shap
import numpy as np

# Make sure your background is a dense 2D array
background_dense = np.asarray(background_tfidf)

class LoggingMasker(shap.maskers.Independent):
    def __init__(self, data, feature_names):
        super().__init__(data)
        self.feature_names = feature_names
        self.masked_vocab_per_call = []

    def __call__(self, mask, *args, **kwargs):
        mask = np.atleast_2d(mask)
        for row in mask:
            idx = np.where(row == 0)[0]
            self.masked_vocab_per_call.append([self.feature_names[i] for i in idx])
        return super().__call__(mask, *args, **kwargs)

# Build masker & explainer (new API)
from shap.explainers import Kernel  # ⬅️ this is the classic Kernel SHAP engine

masker = LoggingMasker(background_dense, feature_names)
f      = lambda x: clf.predict_proba(x)[:, 1]

# explainer = shap.KernelExplainer(f, background_dense)
# explanation = explainer.shap_values(X_explain, nsamples=50)  # legacy interface

explainer = shap.Explainer(f, masker)
explanation = explainer(X_explain, max_evals=1000)



# # SHAP values
# shap_values = explainer.shap_values(X_explain, nsamples=50)
# print("shap_values shape:", np.array(shap_values).shape)


# # For binary classification, shap_values is a list of length 2
# print("shap_values shape (for each class):", np.array(shap_values).shape)
# print("shap_values for class 1 shape:", np.array(shap_values[1]).shape)
# # print("Expected value for class 1:", explainer.expected_value[1])

# print("Expected value for positive class:", explainer.expected_value)



# masker = LoggingMasker(background_dense, feature_names)
# explainer = shap.KernelExplainer(f, background_dense)
# explanation = explainer.shap_values(X_explain, nsamples=50)  # legacy interface



# print("Masking log length:", len(masker.masked_vocab_per_call))
# for i, masked in enumerate(masker.masked_vocab_per_call[:5]):
#     print(f"Call {i}: masked words -> {masked}")




Hold‑out accuracy: 1.0


IndexError: index 59 is out of bounds for axis 0 with size 46

In [20]:
print("background_dense shape:", background_dense.shape)
print("len(feature_names):", len(feature_names))


background_dense shape: (50, 46)
len(feature_names): 46


In [3]:
import shap
print(shap.__version__)

0.48.0


In [18]:
background_dense.shape

(50, 46)

In [22]:
# ------------------------------------------------------------
# 0. Imports
# ------------------------------------------------------------
import numpy as np
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

# ------------------------------------------------------------
# 1. Synthetic corpus + labels
# ------------------------------------------------------------
positive_templates = [
    "I absolutely love this product",
    "Amazing experience and great quality",
    "Fantastic service, will come again",
    "Totally satisfied with the purchase",
    "Highly recommend to everyone"
]
negative_templates = [
    "I really hate this item",
    "Terrible customer service and quality",
    "Very disappointing experience overall",
    "I regret buying this product",
    "Not worth the money at all"
]

# Build a small corpus of 100 documents (50 pos / 50 neg)
random.seed(42)
docs = [random.choice(positive_templates) for _ in range(50)] + \
       [random.choice(negative_templates) for _ in range(50)]
labels = np.array([1]*50 + [0]*50)     # 1 = positive, 0 = negative

# ------------------------------------------------------------
# 2. TF‑IDF vectorisation
# ------------------------------------------------------------
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(docs)           # sparse CSR matrix
feature_names = vectorizer.get_feature_names_out() # list of token/feature names

# ------------------------------------------------------------
# 3. Train / test split + model
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, labels, test_size=0.2, random_state=42, stratify=labels
)

clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_train)

print("Hold‑out accuracy:",
      accuracy_score(y_test, clf.predict(X_test)))

# ------------------------------------------------------------
# 4. Build background data for KernelSHAP
# ------------------------------------------------------------
rng = np.random.default_rng(0)
bg_idx = rng.choice(X_train.shape[0], size=50, replace=False)
background_tfidf = X_train[bg_idx].toarray()   # convert to dense for SHAP
X_explain = X_test[:3].toarray()               # explain first 3 test points

# Debug: Check dimensions
print(f"Background data shape: {background_tfidf.shape}")
print(f"X_explain shape: {X_explain.shape}")
print(f"Number of feature names: {len(feature_names)}")
print(f"Feature names sample: {feature_names[:5]}")

# ------------------------------------------------------------
# 5. Custom logging masker (with bounds checking)
# ------------------------------------------------------------
class LoggingMasker(shap.maskers.Independent):
    def __init__(self, data, feature_names):
        super().__init__(data)
        self.feature_names = feature_names
        self.masked_vocab_per_call = []
        print(f"Masker initialized with {len(feature_names)} feature names and data shape {data.shape}")

    def __call__(self, mask, *args, **kwargs):
        mask = np.atleast_2d(mask)
        for row in mask:
            idx = np.where(row == 0)[0]
            # Add bounds checking to prevent IndexError
            valid_idx = [i for i in idx if i < len(self.feature_names)]
            if len(valid_idx) != len(idx):
                print(f"Warning: Found {len(idx) - len(valid_idx)} indices out of bounds")
            self.masked_vocab_per_call.append([self.feature_names[i] for i in valid_idx])
        return super().__call__(mask, *args, **kwargs)

# ------------------------------------------------------------
# 6. SHAP explanation using new API
# ------------------------------------------------------------
# Prediction function that returns probabilities for positive class
def model_predict(x):
    return clf.predict_proba(x)[:, 1]

# Create masker and explainer
masker = LoggingMasker(background_tfidf, feature_names)
explainer = shap.Explainer(model_predict, masker)

# Get SHAP values
print("Computing SHAP values...")
explanation = explainer(X_explain, max_evals=100)

# ------------------------------------------------------------
# 7. Extract and display results
# ------------------------------------------------------------
print(f"\nSHAP values shape: {explanation.values.shape}")
print(f"Base values shape: {explanation.base_values.shape}")
print(f"Expected value: {explanation.base_values[0]:.4f}")

# Show SHAP values for each test instance
for i in range(len(X_explain)):
    print(f"\n--- Test instance {i} ---")
    print(f"Predicted probability: {model_predict(X_explain[i:i+1])[0]:.4f}")
    print(f"Base value: {explanation.base_values[i]:.4f}")
    
    # Get top contributing features
    shap_vals = explanation.values[i]
    feature_importance = list(zip(feature_names, shap_vals))
    feature_importance.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print("Top 10 contributing features:")
    for feat, val in feature_importance[:10]:
        if abs(val) > 0.001:  # Only show non-zero contributions
            print(f"  {feat}: {val:.4f}")

# Show masking log
print(f"\nMasking log length: {len(masker.masked_vocab_per_call)}")
print("First 5 masking calls:")
for i, masked in enumerate(masker.masked_vocab_per_call[:5]):
    print(f"  Call {i}: masked {len(masked)} features -> {masked[:5]}{'...' if len(masked) > 5 else ''}")

# ------------------------------------------------------------
# 8. Optional: Create a simple visualization
# ------------------------------------------------------------
try:
    # Show SHAP values for the first test instance
    print(f"\n--- SHAP Summary for Test Instance 0 ---")
    test_doc_idx = 0
    
    # Get the original document text (approximate reconstruction)
    test_features = X_explain[test_doc_idx]
    active_features = [(feature_names[i], test_features[i], explanation.values[test_doc_idx][i]) 
                      for i in range(len(feature_names)) if test_features[i] > 0]
    
    print("Active features in this document:")
    for feat, tfidf_val, shap_val in sorted(active_features, key=lambda x: abs(x[2]), reverse=True):
        print(f"  '{feat}': TF-IDF={tfidf_val:.3f}, SHAP={shap_val:.4f}")
        
except Exception as e:
    print(f"Visualization error: {e}")

print("\nDone!")

Hold‑out accuracy: 1.0
Background data shape: (50, 46)
X_explain shape: (3, 46)
Number of feature names: 46
Feature names sample: ['absolutely' 'absolutely love' 'amazing' 'amazing experience' 'buying']
Masker initialized with 46 feature names and data shape (50, 46)
Computing SHAP values...


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(int64, 1d, C), Literal[int](0), array(int64, 1d, C))
 
There are 16 candidate implementations:
[1m     - Of which 16 did not match due to:
     Overload of function 'setitem': File: <numerous>: Line N/A.
       With argument(s): '(array(int64, 1d, C), int64, array(int64, 1d, C))':[0m
[1m      No match.[0m
[0m
[0m[1mDuring: typing of staticsetitem at /opt/miniconda3/envs/rankingSHAP/lib/python3.10/site-packages/shap/maskers/_tabular.py (240)[0m
[1m
File "../../../../../opt/miniconda3/envs/rankingSHAP/lib/python3.10/site-packages/shap/maskers/_tabular.py", line 240:[0m
[1mdef _delta_masking(
    <source elided>
        dpos = 0
[1m        curr_delta_inds[0] = masks[masks_pos]
[0m        [1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m

In [1]:
# ------------------------------------------------------------
# 0. Imports
# ------------------------------------------------------------
import numpy as np
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from collections import Counter

print(f"SHAP version: {shap.__version__}")

# ------------------------------------------------------------
# 1. Synthetic corpus + labels
# ------------------------------------------------------------
positive_templates = [
    "I absolutely love this product",
    "Amazing experience and great quality",
    "Fantastic service, will come again",
    "Totally satisfied with the purchase",
    "Highly recommend to everyone"
]
negative_templates = [
    "I really hate this item",
    "Terrible customer service and quality",
    "Very disappointing experience overall",
    "I regret buying this product",
    "Not worth the money at all"
]

# Build a small corpus of 100 documents (50 pos / 50 neg)
random.seed(42)
docs = [random.choice(positive_templates) for _ in range(50)] + \
       [random.choice(negative_templates) for _ in range(50)]
labels = np.array([1]*50 + [0]*50)     # 1 = positive, 0 = negative

print(f"Created {len(docs)} documents")

# ------------------------------------------------------------
# 2. TF‑IDF vectorisation
# ------------------------------------------------------------
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(docs)           # sparse CSR matrix
feature_names = vectorizer.get_feature_names_out() # list of token/feature names

print(f"TF-IDF matrix shape: {X_tfidf.shape}")
print(f"Number of features: {len(feature_names)}")

# ------------------------------------------------------------
# 3. Train / test split + model
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, labels, test_size=0.2, random_state=42, stratify=labels
)

clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

print("Hold‑out accuracy:", accuracy_score(y_test, clf.predict(X_test)))

# ------------------------------------------------------------
# 4. Prepare data for SHAP
# ------------------------------------------------------------
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

rng = np.random.default_rng(42)
n_background = min(30, X_train_dense.shape[0])
bg_idx = rng.choice(X_train_dense.shape[0], size=n_background, replace=False)
background_data = X_train_dense[bg_idx]

n_explain = min(3, X_test_dense.shape[0])
X_explain = X_test_dense[:n_explain]

print(f"Background data shape: {background_data.shape}")
print(f"Data to explain shape: {X_explain.shape}")

# ------------------------------------------------------------
# 5. Wrapper-based Logging (avoids Numba issues)
# ------------------------------------------------------------
class MaskingLogger:
    """A separate logger that doesn't interfere with SHAP's internal operations"""
    
    def __init__(self, feature_names):
        self.feature_names = np.array(feature_names)
        self.prediction_calls = []
        self.call_count = 0
        
    def log_prediction_call(self, X):
        """Log which features are active (non-zero) in each prediction call"""
        self.call_count += 1
        
        for i, row in enumerate(X):
            # Find active features (non-zero values)
            active_indices = np.where(row != 0)[0]
            inactive_indices = np.where(row == 0)[0]
            
            active_features = [self.feature_names[idx] for idx in active_indices if idx < len(self.feature_names)]
            inactive_features = [self.feature_names[idx] for idx in inactive_indices if idx < len(self.feature_names)]
            
            self.prediction_calls.append({
                'call_number': self.call_count,
                'sample_in_batch': i,
                'active_features': active_features,
                'inactive_features': inactive_features,  # These are the "masked" features
                'n_active': len(active_features),
                'n_inactive': len(inactive_features)
            })
    
    def get_summary(self):
        """Get summary statistics about masking patterns"""
        if not self.prediction_calls:
            return "No prediction calls logged"
        
        # Count how often each feature was masked (inactive)
        all_masked = []
        all_active = []
        
        for call in self.prediction_calls:
            all_masked.extend(call['inactive_features'])
            all_active.extend(call['active_features'])
        
        masked_counts = Counter(all_masked)
        active_counts = Counter(all_active)
        
        avg_masked = np.mean([call['n_inactive'] for call in self.prediction_calls])
        avg_active = np.mean([call['n_active'] for call in self.prediction_calls])
        
        return {
            'total_prediction_calls': len(self.prediction_calls),
            'avg_features_masked': avg_masked,
            'avg_features_active': avg_active,
            'most_frequently_masked': masked_counts.most_common(10),
            'most_frequently_active': active_counts.most_common(10),
            'sample_calls': self.prediction_calls[:5]  # First 5 calls for inspection
        }

# Create the logger
masking_logger = MaskingLogger(feature_names)

# ------------------------------------------------------------
# 6. Wrapped prediction function with logging
# ------------------------------------------------------------
def predict_proba_positive_with_logging(X):
    """Prediction function that logs masking patterns"""
    # Log the call pattern
    masking_logger.log_prediction_call(X)
    
    # Make actual prediction
    return clf.predict_proba(X)[:, 1]

# Test the logging function
print("\nTesting logging function...")
test_pred = predict_proba_positive_with_logging(X_explain[:1])
print(f"Test prediction: {test_pred[0]:.4f}")

# ------------------------------------------------------------
# 7. SHAP Explanation with standard masker + logging
# ------------------------------------------------------------
print("\nCreating SHAP explainer with standard masker + logging wrapper...")

# Use standard masker (no Numba conflicts)
masker = shap.maskers.Independent(background_data)
explainer = shap.Explainer(predict_proba_positive_with_logging, masker)

print("Computing SHAP values (this will log all masking patterns)...")
explanation = explainer(X_explain, max_evals=100)

shap_values = explanation.values
base_values = explanation.base_values

print(f"✅ Success! SHAP values shape: {shap_values.shape}")

# ------------------------------------------------------------
# 8. Analyze SHAP results
# ------------------------------------------------------------
print("\n" + "="*60)
print("SHAP ANALYSIS RESULTS")
print("="*60)

for i in range(len(X_explain)):
    print(f"\n--- Test Instance {i} ---")
    
    # Note: we need to use the original function for final prediction to avoid extra logging
    pred_prob = clf.predict_proba(X_explain[i:i+1])[:, 1][0]
    base_val = base_values[i] if hasattr(base_values, '__len__') else base_values
    
    print(f"Predicted probability: {pred_prob:.4f}")
    print(f"Base value: {base_val:.4f}")
    print(f"SHAP sum: {base_val + np.sum(shap_values[i]):.4f}")
    
    # Get top contributing features
    feature_contribs = [(feature_names[j], shap_values[i][j], X_explain[i][j]) 
                       for j in range(len(feature_names)) if abs(shap_values[i][j]) > 0.001]
    
    feature_contribs.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print(f"Top contributing features:")
    for feat_name, shap_val, tfidf_val in feature_contribs[:8]:
        direction = "→ POS" if shap_val > 0 else "→ NEG"
        print(f"  '{feat_name}': {shap_val:+.4f} {direction} (TF-IDF: {tfidf_val:.3f})")

# ------------------------------------------------------------
# 9. Analyze masking patterns
# ------------------------------------------------------------
print(f"\n" + "="*60)
print("MASKING PATTERN ANALYSIS")
print("="*60)

summary = masking_logger.get_summary()

if isinstance(summary, dict):
    print(f"Total prediction calls during SHAP: {summary['total_prediction_calls']}")
    print(f"Average features masked per call: {summary['avg_features_masked']:.1f}")
    print(f"Average features active per call: {summary['avg_features_active']:.1f}")
    
    print(f"\nMost frequently MASKED features (these have high importance):")
    for feat, count in summary['most_frequently_masked'][:10]:
        percentage = (count / summary['total_prediction_calls']) * 100
        print(f"  '{feat}': masked {count} times ({percentage:.1f}%)")
    
    print(f"\nMost frequently ACTIVE features (these are often in coalitions):")
    for feat, count in summary['most_frequently_active'][:10]:
        percentage = (count / summary['total_prediction_calls']) * 100
        print(f"  '{feat}': active {count} times ({percentage:.1f}%)")
    
    print(f"\nSample prediction calls (showing masking patterns):")
    for i, call in enumerate(summary['sample_calls']):
        print(f"  Call {call['call_number']}: {call['n_active']} active, {call['n_inactive']} masked")
        if call['inactive_features']:
            print(f"    Masked: {call['inactive_features'][:3]}{'...' if len(call['inactive_features']) > 3 else ''}")

else:
    print(summary)

# ------------------------------------------------------------
# 10. Global feature importance
# ------------------------------------------------------------
print(f"\n" + "="*60)
print("GLOBAL FEATURE IMPORTANCE")
print("="*60)

global_importance = np.mean(np.abs(shap_values), axis=0)
feature_importance_pairs = list(zip(feature_names, global_importance))
feature_importance_pairs.sort(key=lambda x: x[1], reverse=True)

print("Most important features overall (average |SHAP| value):")
for feat_name, importance in feature_importance_pairs[:12]:
    print(f"  {feat_name}: {importance:.4f}")

print(f"\nDone! ✅")
print(f"Logged {masking_logger.call_count} prediction calls during SHAP explanation.")

/opt/miniconda3/envs/rankingSHAP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SHAP version: 0.45.1
Created 100 documents
TF-IDF matrix shape: (100, 46)
Number of features: 46
Hold‑out accuracy: 1.0
Background data shape: (30, 46)
Data to explain shape: (3, 46)

Testing logging function...
Test prediction: 0.8024

Creating SHAP explainer with standard masker + logging wrapper...
Computing SHAP values (this will log all masking patterns)...
✅ Success! SHAP values shape: (3, 46)

SHAP ANALYSIS RESULTS

--- Test Instance 0 ---
Predicted probability: 0.8024
Base value: 0.5215
SHAP sum: 0.8024
Top contributing features:
  'highly': +0.0849 → POS (TF-IDF: 0.577)
  'highly recommend': +0.0767 → POS (TF-IDF: 0.577)
  'recommend': +0.0762 → POS (TF-IDF: 0.577)
  'hate item': +0.0099 → POS (TF-IDF: 0.000)
  'hate': +0.0098 → POS (TF-IDF: 0.000)
  'amazing': -0.0095 → NEG (TF-IDF: 0.000)
  'great quality': -0.0095 → NEG (TF-IDF: 0.000)
  'experience great': -0.0094 → NEG (TF-IDF: 0.000)

--- Test Instance 1 ---
Predicted probability: 0.8158
Base value: 0.5215
SHAP sum: 0.81

In [5]:
import numpy as np
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re

# Sample documents and labels
docs = [
    "the cat sat on the mat",
    "the dog lay on the rug",
    "the bird flew over the tree",
    "the fish swam in the pond"
]
labels = [0, 0, 1, 1]

# Fit TF-IDF
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names_out()

# Fit a simple model
clf = LogisticRegression().fit(X, labels)

# Custom masker: replaces masked words with <unk> before vectorizing
class UnkTokenMasker(shap.maskers.Independent):
    def __init__(self, data, feature_names, docs, vectorizer):
        super().__init__(data)
        self.feature_names = feature_names
        self.docs = docs
        self.vectorizer = vectorizer

    def __call__(self, mask, *args, **kwargs):
        mask = np.atleast_2d(mask)
        new_X = []
        for i, mask_row in enumerate(mask):
            masked_indices = np.where(mask_row == 0)[0]
            masked_words = set(self.feature_names[masked_indices])
            doc = self.docs[i % len(self.docs)]
            # Replace masked words with <unk>
            for word in masked_words:
                doc = re.sub(r'\b{}\b'.format(re.escape(word)), '<unk>', doc)
            x_new = self.vectorizer.transform([doc]).toarray()[0]
            new_X.append(x_new)
        return np.array(new_X)

# Add <unk> to vocabulary and re-fit
docs_with_unk = docs
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", vocabulary=None)
X = vectorizer.fit_transform(docs_with_unk)
feature_names = vectorizer.get_feature_names_out()
clf = LogisticRegression().fit(X[:len(docs)], labels)

# Test the masker and explainer
masker = UnkTokenMasker(X[:len(docs)], feature_names, docs, vectorizer)
explainer = shap.KernelExplainer(clf.predict_proba, masker = masker)
shap_values = explainer.shap_values(X[:2], nsamples=5)  # Just test first two docs

print("SHAP values shape:", np.array(shap_values).shape)
print("Done.")

TypeError: KernelExplainer.__init__() missing 1 required positional argument: 'data'

In [4]:
import numpy as np
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re

# Sample documents and labels
docs = [
    "the cat sat on the mat",
    "the dog lay on the rug",
    "the bird flew over the tree",
    "the fish swam in the pond"
]
labels = [0, 0, 1, 1]

# Ensure '<unk>' is in the vocabulary
docs_with_unk = docs + ["<unk>"]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(docs_with_unk)
feature_names = vectorizer.get_feature_names_out()
unk_idx = np.where(feature_names == "<unk>")[0][0]

# Train model
clf = LogisticRegression().fit(X[:len(docs)], labels)

# Custom masker: replaces masked words with <unk> before vectorizing
class UnkTokenMasker:
    def __init__(self, docs, feature_names, vectorizer):
        self.docs = docs
        self.feature_names = feature_names
        self.vectorizer = vectorizer

    def __call__(self, mask):
        # Always process a batch of masks
        mask = np.atleast_2d(mask)
        out = []
        for i, mask_row in enumerate(mask):
            masked_indices = np.where(mask_row == 0)[0]
            masked_words = set(self.feature_names[masked_indices])
            doc = self.docs[i % len(self.docs)]
            for word in masked_words:
                doc = re.sub(r'\b{}\b'.format(re.escape(word)), '<unk>', doc)
            tfidf_vec = self.vectorizer.transform([doc]).toarray()[0]
            out.append(tfidf_vec)
        out = np.array(out)
        if out.shape[0] == 1:
            return out[0]
        return out

# Explain first two docs
masker = UnkTokenMasker(docs, feature_names, vectorizer)
explainer = shap.KernelExplainer(clf.predict_proba, masker=masker)
shap_values = explainer.shap_values(X[:2], nsamples=5)

print("SHAP values shape:", np.array(shap_values).shape)
print("Done.")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import kendalltau
from utils.helper_functions import rank_list

def bm25_based_model_predict(
    array,
    original_model_predict,
    query_features,
    bm25_features_mask,  # New parameter to identify BM25 features
    unk_token="<unk>",
    similarity_coefficient=lambda x, y: kendalltau(x, y)[0],
):
    """
    Custom model predict function for NLP models with BM25 and sentence embeddings.
    
    Args:
        array: Perturbation array from SHAP
        original_model_predict: Your NLP model's predict function
        query_features: Original query features (documents)
        bm25_features_mask: Boolean mask indicating which features are BM25-based
        unk_token: Token to use for masking (default: "<unk>")
        similarity_coefficient: Function to compare rankings
    """
    # Get original prediction and ranking
    pred = original_model_predict(query_features)
    og_rank = rank_list(pred)
    
    scores = []
    
    for perturbation in array:
        # Create masked features based on BM25 perturbations
        masked_features = []
        
        for doc_idx, doc_features in enumerate(query_features):
            masked_doc = doc_features.copy()
            
            # Apply masking only to BM25-based features
            for feature_idx, should_mask in enumerate(perturbation):
                if pd.isna(should_mask) and bm25_features_mask[feature_idx]:
                    # Replace BM25 feature with <unk> token
                    if isinstance(masked_doc[feature_idx], str):
                        # For text features, replace with <unk>
                        masked_doc[feature_idx] = unk_token
                    else:
                        # For numerical BM25 scores, set to 0 or background value
                        masked_doc[feature_idx] = 0.0
                        
            masked_features.append(masked_doc)
        
        # Get prediction with masked features
        masked_pred = original_model_predict(masked_features)
        masked_rank = rank_list(masked_pred)
        
        # Calculate similarity between original and masked rankings
        similarity_score = similarity_coefficient(og_rank, masked_rank)
        scores.append(similarity_score)
    
    return np.array(scores)

class CustomNLPRankingShap:
    """
    Custom RankingShap implementation for NLP models with BM25 and HNSW.
    """
    
    def __init__(
        self,
        nlp_model,
        background_data,
        bm25_feature_indices,
        unk_token="<unk>",
        explanation_size=5,
        nsamples="auto",
        rank_similarity_coefficient=lambda x, y: kendalltau(x, y)[0]
    ):
        """
        Initialize custom NLP RankingShap explainer.
        
        Args:
            nlp_model: Your NLP model with BM25 and sentence embeddings
            background_data: Background dataset for masking
            bm25_feature_indices: List of indices corresponding to BM25 features
            unk_token: Token to use for masking
            explanation_size: Number of top features to explain
            nsamples: Number of samples for SHAP
            rank_similarity_coefficient: Function to compare rankings
        """
        self.nlp_model = nlp_model
        self.background_data = background_data
        self.unk_token = unk_token
        self.explanation_size = explanation_size
        self.nsamples = nsamples
        self.rank_similarity_coefficient = rank_similarity_coefficient
        
        # Create boolean mask for BM25 features
        self.feature_shape = np.shape(background_data[0])
        self.num_features = len(background_data[0])
        self.bm25_features_mask = np.zeros(self.num_features, dtype=bool)
        self.bm25_features_mask[bm25_feature_indices] = True
        
        # Initialize SHAP explainer with custom predict function
        import shap
        from functools import partial
        
        self.custom_predict = partial(
            bm25_based_model_predict,
            original_model_predict=self.nlp_model,
            bm25_features_mask=self.bm25_features_mask,
            unk_token=self.unk_token,
            similarity_coefficient=self.rank_similarity_coefficient
        )
        
        self.explainer = shap.KernelExplainer(
            self.placeholder_predict, 
            self.background_data, 
            nsamples=self.nsamples
        )
    
    def placeholder_predict(self, array):
        """Placeholder function that will be replaced during explanation."""
        raise ValueError("Model predict function not initialized for current query")
    
    def explain_query(self, query_features, query_id=""):
        """
        Generate explanation for a specific query.
        
        Args:
            query_features: Features for the query documents
            query_id: Optional identifier for the query
            
        Returns:
            Dictionary with feature importance scores
        """
        from shap.utils._legacy import convert_to_model
        from functools import partial
        
        # Set query-specific model predict function
        self.explainer.model = convert_to_model(
            partial(self.custom_predict, query_features=query_features)
        )
        
        # Create vector of None values for SHAP
        vector_of_nones = np.array([np.full(self.feature_shape, None)])
        
        # Get SHAP values
        shap_values = self.explainer.shap_values(vector_of_nones, nsamples=self.nsamples)[0]
        
        # Convert to explanation dictionary
        explanation = {
            f"feature_{i+1}": shap_values[i] 
            for i in range(len(shap_values))
        }
        
        return explanation

In [9]:
array = np.array([123., 123.,  np.nan, 123., 123., 123., 123., 123., 123., 123., 123.,
    123., 123., 123., 123., 123., 123., 123.,  np.nan,  np.nan, 123., 123.,
    123., 123., 123., 123., 123., 123., 123., 123., 123.,  np.nan, 123.,
    123., 123., 123.,  np.nan, 123., 123., 123., 123., 123., 123., 123.,
    123., 123.])


array_str = np.where(array == 123., '<unk>', array.astype(str))
print(array_str)



['<unk>' '<unk>' 'nan' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>'
 '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>'
 'nan' 'nan' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>'
 '<unk>' '<unk>' '<unk>' '<unk>' 'nan' '<unk>' '<unk>' '<unk>' '<unk>'
 'nan' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>' '<unk>'
 '<unk>']


In [ ]:
def new_model_predict_val(
    array,
    original_model_predict,
    query_features,
    documents,
    similarity_coefficient=lambda x, y: kendalltau(x, y)[0],
    mixed_type_input=False,
):


    # Adjust feature vectors: substitute not None features for all candidate documents with background/0 value
    # Adjust feature vectors: substitute not None features for all candidate documents with background/0 value
    if not mixed_type_input:
        adjusted_features = np.array(
            [[np.where(pd.isna(a), doc, a) for doc in query_features] for a in array]
        )
    else:
        adjusted_features = np.array(
            [
                [pd.Series(doc).where(pd.isna(a), a).values for doc in query_features]
                for a in array
            ]
        )  # Need to use pd for mixed type array.

    print(adjusted_features)